In [4]:
%matplotlib inline
import itertools
from time import time
import csv
import dask
from dask.diagnostics import ProgressBar

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from kifmm_py import KiFmm

# Contents
- [Single Precision](#single-precision)
- [Double Precision](#double-precision)

# Single Precision

In [2]:
@dask.delayed
def setup_fmm_blas(sources, targets, charges, kernel_eval_type, kernel, field_translation, surface_diff, svd_threshold, depth, expansion_order):
    tmp = [expansion_order] * (depth + 1)

    start_time = time()
    fmm = KiFmm(
        tmp,
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        prune_empty=True,
        timed=True,
        svd_threshold=svd_threshold,
        surface_diff=surface_diff,
        depth=depth
    )
    setup_time = time() - start_time
    return fmm, setup_time


### BLAS Field Translations

In [ ]:
np.random.seed(0)
surface_diff_vec = [0, 1, 2]
svd_threshold_vec = [None, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 2e-2]
depth_vec = [4, 5]
expansion_order_vec = [3, 4, 5, 6]

parameters = list(itertools.product(surface_diff_vec, svd_threshold_vec, depth_vec, expansion_order_vec))

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
n_vec = 1
n_crit = None
depth = 3
n_sources = 1000000
n_targets = 1000000
kernel = "laplace"
field_translation = "blas"
kernel_eval_type = (
    "eval"
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []

fmms = []
for (i, (surface_diff, svd_threshold, depth, expansion_order)) in enumerate(parameters):
    fmm = setup_fmm_blas(
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        surface_diff,
        svd_threshold,
        depth,
        expansion_order
    )
    fmms.append(fmm)

with ProgressBar():
    fmms = dask.compute(fmms)


for i, fmm in enumerate(fmms[0]):
    fmm = fmm[0]
    s = time()
    fmm.evaluate()
    times.append(time()-s)
    print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")

    leaf = fmm.target_leaves[1]
    found = fmm.potentials(leaf)[0]
    targets_leaf = fmm.target_coordinates(leaf)
    expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

    relative_error = np.abs(expected-found)/expected

    max_relative_error = np.max(relative_error)
    min_relative_error = np.min(relative_error)
    mean_relative_error = np.mean(relative_error)

    rel_error.append((min_relative_error, mean_relative_error, max_relative_error))


with open('accuracy_single_precision_blas.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['surface_diff', 'svd_threshold', 'depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
    for row in zip(parameters, times, rel_error):
        writer.writerow([row[0][0], row[0][1], row[0][2], row[0][3], row[1], row[2][0], row[2][1], row[2][2]])

In [ ]:
fmms

### FFT Field Translations

In [6]:
@dask.delayed
def setup_fmm_fft(sources, targets, charges, kernel_eval_type, kernel, field_translation, depth, expansion_order):
    tmp = [expansion_order] * (depth + 1)

    start_time = time()
    fmm = KiFmm(
        tmp,
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        prune_empty=True,
        timed=True,
        depth=depth
    )
    setup_time = time() - start_time
    return fmm, setup_time


In [5]:
np.random.seed(0)

depth_vec = [4, 5]
expansion_order_vec = [3, 4, 5, 6, 7, 8]

parameters = list(itertools.product(depth_vec, expansion_order_vec))

np.random.seed(0)

dim = 3
dtype = np.float32
ctype = np.complex64

# Set FMM Parameters
n_vec = 1
n_crit = None
n_sources = 1000000
n_targets = 1000000
kernel = "laplace"
field_translation = "fft"
kernel_eval_type = (
    "eval"
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []


fmms = []
for (i, (depth, expansion_order)) in enumerate(parameters):
    fmm = setup_fmm_fft(
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        depth,
        expansion_order
    )
    fmms.append(fmm)

with ProgressBar():
    fmms = dask.compute(fmms)


for i, fmm in enumerate(fmms[0]):
    fmm = fmm[0]
    s = time()
    fmm.evaluate()
    times.append(time()-s)
    print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")

    leaf = fmm.target_leaves[1]
    found = fmm.potentials(leaf)[0]
    targets_leaf = fmm.target_coordinates(leaf)
    expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

    relative_error = np.abs(expected-found)/expected

    max_relative_error = np.max(relative_error)
    min_relative_error = np.min(relative_error)
    mean_relative_error = np.mean(relative_error)

    rel_error.append((min_relative_error, mean_relative_error, max_relative_error))

with open('accuracy_single_precision_fft.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
    for row in zip(parameters, times, rel_error):
        writer.writerow([row[0][0], row[0][1], row[1], row[2][0], row[2][1], row[2][2]])

[########################################] | 100% Completed | 18.90 s
Computing: 1/12 Eval Time: 0.5576698780059814
Computing: 2/12 Eval Time: 0.6043729782104492
Computing: 3/12 Eval Time: 0.6226398944854736
Computing: 4/12 Eval Time: 0.6437809467315674
Computing: 5/12 Eval Time: 0.6808841228485107
Computing: 6/12 Eval Time: 0.7842469215393066
Computing: 7/12 Eval Time: 0.16890978813171387
Computing: 8/12 Eval Time: 0.25838708877563477
Computing: 9/12 Eval Time: 0.4390740394592285
Computing: 10/12 Eval Time: 0.6755778789520264
Computing: 11/12 Eval Time: 1.0823841094970703
Computing: 12/12 Eval Time: 1.8447151184082031


### Examine Parameters

In [6]:
# blas = pd.read_csv('accuracy_single_precision_blas.csv')
# fft = pd.read_csv('accuracy_single_precision_fft.csv')

# # Filter for max rel error with given number of digits in final solution
# fft_3 = fft[(fft['max_rel_err'] < 1e-3) & (fft['max_rel_err'] > 1e-4)]
# fft_4 = fft[(fft['max_rel_err'] < 1e-4) & (fft['max_rel_err'] > 1e-5)]
# fft_5 = fft[(fft['max_rel_err'] < 1e-5) & (fft['max_rel_err'] > 1e-6)]

# blas_3 = blas[(blas['max_rel_err'] < 1e-3) & (blas['max_rel_err'] > 1e-4)]
# blas_4 = blas[(blas['max_rel_err'] < 1e-4) & (blas['max_rel_err'] > 1e-5)]
# blas_5 = blas[(blas['max_rel_err'] < 1e-5) & (blas['max_rel_err'] > 1e-6)]

# blas_vec = [blas_3, blas_4, blas_5]
# fft_vec = [fft_3, fft_4, fft_5]

# blas_best = []
# fft_best = []

# for (blas, fft) in zip(blas_vec, fft_vec):
#     min_index = blas['time'].idxmin()
#     blas_best.append(blas.loc[min_index])
#     min_index = fft['time'].idxmin()
#     fft_best.append(fft.loc[min_index])

# blas_best_times = []
# fft_best_times = []
# for (blas, fft) in zip(blas_best, fft_best):
#     blas_best_times.append(blas['time'])
#     fft_best_times.append(fft['time'])


# # Bar width
# width = 0.35

# fig, ax = plt.subplots()

# x = np.arange(len(blas_best_times))

# ax.bar(x - width/2, blas_best_times, width, label='BLAS')
# ax.bar(x + width/2, fft_best_times, width, label='FFT')
# ax.set_ylabel('FMM Runtime (s)')
# ax.set_xlabel('Digits of Precision')
# ax.set_xticks([0, 1, 2])
# ax.set_xticklabels([3, 4, 5])
# ax.legend()

# plt.show()

# Double Precision

## BLAS Field Translations

In [ ]:
np.random.seed(0)
surface_diff_vec = [0, 1, 2]
svd_threshold_vec = [None, 1e-15, 1e-12, 1e-9, 1e-6, 1e-3, 1e-1]
depth_vec = [4, 5]
expansion_order_vec = [5, 6, 7, 8, 9, 10]

parameters = list(itertools.product(surface_diff_vec, svd_threshold_vec, depth_vec, expansion_order_vec))

dim = 3
dtype = np.float64
ctype = np.complex128

# Set FMM Parameters
n_vec = 1
n_crit = None
depth = 3
n_sources = 1000000
n_targets = 1000000
kernel = "laplace"
field_translation = "blas"
kernel_eval_type = (
    "eval"
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []

fmms = []
for (i, (surface_diff, svd_threshold, depth, expansion_order)) in enumerate(parameters):
    fmm = setup_fmm_blas(
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        surface_diff,
        svd_threshold,
        depth,
        expansion_order
    )
    fmms.append(fmm)

with ProgressBar():
    fmms = dask.compute(fmms)


for i, fmm in enumerate(fmms[0]):
    fmm = fmm[0]
    s = time()
    fmm.evaluate()
    times.append(time()-s)
    print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")

    leaf = fmm.target_leaves[1]
    found = fmm.potentials(leaf)[0]
    targets_leaf = fmm.target_coordinates(leaf)
    expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

    relative_error = np.abs(expected-found)/expected

    max_relative_error = np.max(relative_error)
    min_relative_error = np.min(relative_error)
    mean_relative_error = np.mean(relative_error)

    rel_error.append((min_relative_error, mean_relative_error, max_relative_error))

with open('accuracy_double_precision_blas.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['surface_diff', 'svd_threshold', 'depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
    for row in zip(parameters, times, rel_error):
        writer.writerow([row[0][0], row[0][1], row[0][2], row[0][3], row[1], row[2][0], row[2][1], row[2][2]])

[                                        ] | 0% Completed | 129.67 us

## FFT Field Translations

In [7]:
np.random.seed(0)

depth_vec = [4, 5]
expansion_order_vec = [5, 6, 7, 8, 9, 10]

parameters = list(itertools.product(depth_vec, expansion_order_vec))

np.random.seed(0)

dim = 3
dtype = np.float64
ctype = np.complex128

# Set FMM Parameters
n_vec = 1
n_crit = None
n_sources = 1000000
n_targets = 1000000
kernel = "laplace"
field_translation = "fft"
kernel_eval_type = (
    "eval"
)

sources = np.reshape(
    np.random.rand(n_sources * dim), (n_sources, dim)
).astype(dtype)
targets = np.reshape(
    np.random.rand(n_targets * dim), (n_targets, dim)
).astype(dtype)
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec)
).astype(dtype)

rel_error = []
times = []
setup = []


fmms = []
for (i, (depth, expansion_order)) in enumerate(parameters):
    fmm = setup_fmm_fft(
        sources,
        targets,
        charges,
        kernel_eval_type,
        kernel,
        field_translation,
        depth,
        expansion_order
    )
    fmms.append(fmm)

with ProgressBar():
    fmms = dask.compute(fmms)


for i, fmm in enumerate(fmms[0]):
    fmm = fmm[0]
    s = time()
    fmm.evaluate()
    times.append(time()-s)
    print(f"Computing: {i+1}/{len(parameters)} Eval Time: {times[i]}")

    leaf = fmm.target_leaves[1]
    found = fmm.potentials(leaf)[0]
    targets_leaf = fmm.target_coordinates(leaf)
    expected = fmm.evaluate_kernel(sources, targets_leaf, charges)

    relative_error = np.abs(expected-found)/expected

    max_relative_error = np.max(relative_error)
    min_relative_error = np.min(relative_error)
    mean_relative_error = np.mean(relative_error)

    rel_error.append((min_relative_error, mean_relative_error, max_relative_error))

with open('accuracy_double_precision_fft.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['depth', 'expansion_order', 'time', 'min_rel_err', 'mean_rel_err', 'max_rel_err'])
    for row in zip(parameters, times, rel_error):
        writer.writerow([row[0][0], row[0][1], row[1], row[2][0], row[2][1], row[2][2]])

[########################################] | 100% Completed | 28.39 s
Computing: 1/12 Eval Time: 1.6353890895843506
Computing: 2/12 Eval Time: 1.6218059062957764
Computing: 3/12 Eval Time: 1.7792227268218994
Computing: 4/12 Eval Time: 1.8667809963226318
Computing: 5/12 Eval Time: 2.2157700061798096
Computing: 6/12 Eval Time: 2.8964059352874756
Computing: 7/12 Eval Time: 1.091400384902954
Computing: 8/12 Eval Time: 2.0246992111206055
Computing: 9/12 Eval Time: 2.837571144104004
Computing: 10/12 Eval Time: 5.3911659717559814
Computing: 11/12 Eval Time: 8.259863138198853
Computing: 12/12 Eval Time: 10.994482040405273


In [65]:
# blas = pd.read_csv('accuracy_double_precision_blas.csv')
# fft = pd.read_csv('accuracy_double_precision_fft.csv')

# # Filter for max rel error with given number of digits in final solution
# fft_6 = fft[(fft['max_rel_err'] < 1e-6) & (fft['max_rel_err'] > 1e-7)]
# fft_7 = fft[(fft['max_rel_err'] < 1e-7) & (fft['max_rel_err'] > 1e-8)]
# fft_8 = fft[(fft['max_rel_err'] < 1e-8) & (fft['max_rel_err'] > 1e-9)]
# fft_9 = fft[(fft['max_rel_err'] < 1e-9) & (fft['max_rel_err'] > 1e-10)]
# fft_10 = fft[(fft['max_rel_err'] < 1e-10) & (fft['max_rel_err'] > 1e-11)]


# blas_6 = blas[(blas['max_rel_err'] < 1e-6) & (blas['max_rel_err'] > 1e-7)]
# blas_7 = blas[(blas['max_rel_err'] < 1e-7) & (blas['max_rel_err'] > 1e-8)]
# blas_8 = blas[(blas['max_rel_err'] < 1e-8) & (blas['max_rel_err'] > 1e-9)]
# blas_9 = blas[(blas['max_rel_err'] < 1e-9) & (blas['max_rel_err'] > 1e-10)]
# blas_10 = blas[(blas['max_rel_err'] < 1e-10) & (blas['max_rel_err'] > 1e-11)]


# blas_vec = [blas_6, blas_7, blas_8, blas_9, blas_10]
# fft_vec = [fft_6, fft_7, fft_8, fft_9, fft_10]

# blas_best = []
# fft_best = []

# for (blas, fft) in zip(blas_vec, fft_vec):
#     min_index = blas['time'].idxmin()
#     blas_best.append(blas.loc[min_index])
#     min_index = fft['time'].idxmin()
#     fft_best.append(fft.loc[min_index])

# blas_best_times = []
# fft_best_times = []
# for (blas, fft) in zip(blas_best, fft_best):
#     blas_best_times.append(blas['time'])
#     fft_best_times.append(fft['time'])

In [ ]:
# # Bar width
# width = 0.35

# fig, ax = plt.subplots()

# x = np.arange(len(blas_best_times))

# ax.bar(x - width/2, blas_best_times, width, label='BLAS')
# ax.bar(x + width/2, fft_best_times, width, label='FFT')
# ax.set_ylabel('FMM Runtime (s)')
# ax.set_xlabel('Digits of Precision')
# ax.set_xticks([0, 1, 2, 3, 4])
# ax.set_xticklabels([6, 7, 8, 9, 10])
# ax.legend()

# plt.show()